
---

### Reminder: This 📘 `Python` notebook can be run from VS Code with [these prerequisites](../PREREQS.md).

#### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# Recipe V: 🍋 Connectors w/ Bing
## 🧑‍🍳 Cook using fresher context with live data from APIs

## Step 1: Instantiate a 🔥 kernel 

In [ ]:
!python -m pip install semantic-kernel==0.3.3.dev

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

## Step 2: Integrate live data with the Bing Web Search API 🍋 connector

To get a Bing Web Search API key visit [this resource](https://www.microsoft.com/en-us/bing/apis/bing-web-search-api) to start. Once you have that in hand, you can now grab live search information to get added to the context of your choice. For example, we can find out what's the tallest building in Europe:

In [2]:
import os
from dotenv import load_dotenv
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_skills import WebSearchEngineSkill

load_dotenv()

BING_API_KEY = "75037f51137a43a8bacb6ba35a7203a3"
connector = BingConnector(api_key=os.getenv("BING_API_KEY"))
web_skill = kernel.import_skill(WebSearchEngineSkill(connector), "WebSearch")

search_async = web_skill["searchAsync"]

bing_result = await search_async.invoke_async("What's the tallest building in NYC?")
print(bing_result)

['The tallest building in New York is One World Trade Center, which rises 1,776 feet (541 m). [2] [3] [4] The 104-story [A] skyscraper also stands as the tallest building in the United States, the tallest building in the Western Hemisphere, and the seventh-tallest building in the world.']


> ✅ Keep in mind that getting a Bing Web Search API key will require you to have a fresh FREE Azure account, or else a paid Azure account with active billing credentials.

> 🤔 **Get `"Error: Response status code does not indicate success: 401 (PermissionDenied)."` message?** This could happen if you just provisioned a Bing Web Search API. Wait a few minutes and try again. 

> 😱 **Get a different error message?** go to https://aka.ms/sk/discord where we have realtime support available to troubleshoot your problem.

## Step 3: Combine existing 🥑 memories with fresh 🍋 data

So we take our memories to the next level by providing it "live" memories that are delivered via a connector. It's a terrific combination of both _timeless_ information and _timely_ information. We feed the output of the `SearchAsync` function into a custom-crafted inline semantic function to tell a short story while including a few "facts" as inline 🥑 memories, like:

1. I'm tall-ish: about 5ft 11in
2. I like sushi — but not the kind with uni

And we'll add a fresh memory that we grab from a Bing 🍋 connector to brag that we've been to `the tallest building in NYC` by adding a 3rd fact of:

3. I've been to <the tallest building in NYC>

In [3]:
skprompt = """
    These are facts about me:
    1. I'm tall-ish: about 5ft 11in
    2. I like sushi — but not the kind with uni
    3. I've been to {{$input}}
    Here is a short story under 100 words about me:
"""
my_inline_semantic_function = kernel.create_semantic_function(skprompt, max_tokens=200, temperature=0.1, top_p=0.1)

result = await kernel.run_async(search_async, my_inline_semantic_function, input_str="What's the tallest building in NYC?")

print(result)


I'm a tall-ish person who loves sushi, but not the kind with uni. I've been to the tallest building in New York, One World Trade Center, which stands at 1,776 feet. It's the tallest building in the United States, the tallest building in the Western Hemisphere, and the seventh-tallest building in the world. I'm always looking for new and exciting places to explore, and I'm always up for a challenge.


Now you can see how it's possible to use Semantic Kernel in a variety of ways — from designing prompts, expanding a prompt's context with memories, and now drawing live data into an LLM AI's processing capability.

## Step 4: Let's go one step further and add some extra seasoning with the `SummarizeSkill.Notegen` 🧂 function

In [15]:
skills_directory = "./skills/"
mySkill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")

result = await kernel.run_async(
    search_async,
    my_inline_semantic_function,
    mySkill["Notegen"],
    input_str="What's the tallest building in NYC?"
)

print(result)

Personal Info
- Tall-ish
- Likes sushi, not uni
- Visited One World Trade Center (1,776 ft)
- Always exploring, up for challenge


If you look closely at the code, what we did is make a "chain" of functions. We'll be doing that in our next recipe section in the context of a well-known process for innovation called "Design Thinking."

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Design thinking with 🔥 kernel is up next!](../e6-design-chain/notebook.ipynb)

Or stay a longer while and ask Bing to added new information from the Internet to fortify your LLM AI magic. And you can also try different functions within the `SummarizeSkill` to see different outputs in Step 4.